In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from blitz.modules import BayesianLinear
from blitz.utils import variational_estimator

from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X, y = load_boston(return_X_y=True)
X = StandardScaler().fit_transform(X)
y = StandardScaler().fit_transform(np.expand_dims(y, -1))

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.25,
                                                    random_state=42)


X_train, y_train = torch.tensor(X_train).float(), torch.tensor(y_train).float()
X_test, y_test = torch.tensor(X_test).float(), torch.tensor(y_test).float()


@variational_estimator
class BayesianRegressor(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        #self.linear = nn.Linear(input_dim, output_dim)
        self.blinear1 = BayesianLinear(input_dim, 512)
        self.blinear2 = BayesianLinear(512, output_dim)
        
    def forward(self, x):
        x_ = self.blinear1(x)
        x_ = F.relu(x_)
        return self.blinear2(x_)


def evaluate_regression(regressor,
                        X,
                        y,
                        samples = 100,
                        std_multiplier = 2):
    preds = [regressor(X) for i in range(samples)]
    preds = torch.stack(preds)
    means = preds.mean(axis=0)
    stds = preds.std(axis=0)
    ci_upper = means + (std_multiplier * stds)
    ci_lower = means - (std_multiplier * stds)
    ic_acc = (ci_lower <= y) * (ci_upper >= y)
    ic_acc = ic_acc.float().mean()
    return ic_acc, (ci_upper >= y).float().mean(), (ci_lower <= y).float().mean()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
regressor = BayesianRegressor(13, 1).to(device)
optimizer = optim.Adam(regressor.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

ds_train = torch.utils.data.TensorDataset(X_train, y_train)
dataloader_train = torch.utils.data.DataLoader(ds_train, batch_size=16, shuffle=True)

ds_test = torch.utils.data.TensorDataset(X_test, y_test)
dataloader_test = torch.utils.data.DataLoader(ds_test, batch_size=16, shuffle=True)




c:\Users\luke\anaconda3\envs\torch_nish\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California ho

CI acc: 0.01, CI upper acc: 0.68, CI lower acc: 0.33
Loss: 77.3148
CI acc: 0.04, CI upper acc: 0.73, CI lower acc: 0.31
Loss: 69.4747
CI acc: 0.06, CI upper acc: 0.63, CI lower acc: 0.43
Loss: 61.7484
CI acc: 0.09, CI upper acc: 0.60, CI lower acc: 0.50
Loss: 54.0158
CI acc: 0.13, CI upper acc: 0.78, CI lower acc: 0.35
Loss: 46.5165
CI acc: 0.24, CI upper acc: 0.72, CI lower acc: 0.53
Loss: 38.8187
CI acc: 0.27, CI upper acc: 0.75, CI lower acc: 0.52
Loss: 31.4939
CI acc: 0.42, CI upper acc: 0.74, CI lower acc: 0.68
Loss: 24.6717
CI acc: 0.54, CI upper acc: 0.83, CI lower acc: 0.71
Loss: 17.8546
CI acc: 0.54, CI upper acc: 0.96, CI lower acc: 0.57
Loss: 12.5065
CI acc: 0.87, CI upper acc: 0.96, CI lower acc: 0.91
Loss: 7.6664
CI acc: 0.96, CI upper acc: 0.97, CI lower acc: 0.99
Loss: 5.2111
CI acc: 0.91, CI upper acc: 0.98, CI lower acc: 0.92
Loss: 2.9341
CI acc: 0.94, CI upper acc: 0.94, CI lower acc: 1.00
Loss: 2.4700
CI acc: 0.98, CI upper acc: 0.98, CI lower acc: 0.99
Loss: 2.3046


KeyboardInterrupt: 

In [13]:
for i, (datapoints, labels) in enumerate(dataloader_train):
    datapoints
labels


tensor([[-0.4172],
        [ 1.0304],
        [-0.8416],
        [ 0.9869],
        [-0.2104],
        [-1.0049],
        [ 0.4427],
        [-1.1572],
        [-0.2648],
        [ 2.9895],
        [ 0.4971]])

In [ ]:
iteration = 0
for epoch in range(1000):
    for i, (datapoints, labels) in enumerate(dataloader_train):
        optimizer.zero_grad()
        
        loss = regressor.sample_elbo(inputs=datapoints.to(device),
                           labels=labels.to(device),
                           criterion=criterion,
                           sample_nbr=3,
                           complexity_cost_weight=1/X_train.shape[0])
        loss.backward()
        optimizer.step()
        
        iteration += 1
        if iteration%100==0:
            ic_acc, under_ci_upper, over_ci_lower = evaluate_regression(regressor,
                                                                        X_test.to(device),
                                                                        y_test.to(device),
                                                                        samples=25,
                                                                        std_multiplier=3)
            
            print("CI acc: {:.2f}, CI upper acc: {:.2f}, CI lower acc: {:.2f}".format(ic_acc, under_ci_upper, over_ci_lower))
            print("Loss: {:.4f}".format(loss))
